In [1]:
import torch
import torchvision
import torch.onnx

import ipywidgets as widgets
from IPython.display import display

ModuleNotFoundError: No module named 'torch'

In [7]:
w_1 = widgets.RadioButtons(
    options=['CPU', 'GPU'],
    description='select device:',
    disabled=False
)
w_2 = widgets.Dropdown(
    options=['trace', 'script', 'onnx'],
    value='trace',
    description='method:',
    disabled=False,
)
w_3 = widgets.Text(
    value='export_model',
    placeholder='file name (no file extension needed)',
    description='file_name:',
    disabled=False
)

tab_contents = ['batch size', 'rgb ch', 'width', 'hight']
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = [str(i) for i in range(len(children))]


left_box = widgets.VBox([w_1, w_2])
right_box = widgets.VBox([w_3, tab])
widgets.HBox([left_box, right_box])

In [21]:
if len(tab.children[0].value) != 0:
    batch_size = int(tab.children[0].value)
else: 
    batch_size = None
if len(tab.children[1].value) != 0:
    ch_rgb = int(tab.children[1].value)
else:
    print("ERROR empty ch value")
if len(tab.children[2].value) != 0:
    ch_w = int(tab.children[2].value)
else:
    print("ERROR empty ch value")
if len(tab.children[3].value) != 0:
    ch_h = int(tab.children[3].value)
else:
    print("ERROR empty ch value")

ERROR empty ch value
ERROR empty ch value
ERROR empty ch value


In [47]:
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

if w_1.value == 'GPU':
    if device == 'cuda':
        model = model.to(device)
    else:
        print("NO GPU DEVICE DETECTED")

model.eval()


if w_2.value == 'script':

    with torch.no_grad():
        traced_module = torch.jit.script(model)
        
    #save model    
    traced_module.save(w_3.value+'.pt')
    print("exported as script!")
    
elif w_2.value == 'trace':
    
    # creates input example
    if w_1.value == 'GPU' and device == 'cuda':
        x_trace = torch.rand((1,3, 300, 400),device=device)
    else:
        x_trace = torch.rand(1,3, 300, 400)

    with torch.no_grad():
        traced_module = torch.jit.trace(model, x_trace)
    
    #save model
    traced_module.save(w_3.value+'.pt')
    print("exported as trace!")
    
else:
    
    # creates input example
    if w_1.value == 'GPU' and device == 'cuda':
        x_onnx = torch.rand((3, 300, 400),device=device)
    else:
        x_onnx = [torch.rand(3, 1200, 800)]
        x_onnx = [torch.randn((3, 1200, 800), device = 'cpu')]
    
    #save model
    torch.onnx.export(model, x_onnx, w_3.value + '.onnx' ,verbose=True, do_constant_folding=True, opset_version = 11)
    print("exported as onnx!")

graph(%image.1 : Float(3, 1200, 800, strides=[960000, 800, 1], requires_grad=0, device=cpu),
      %backbone.body.conv1.weight : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %backbone.body.layer1.0.conv1.weight : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %backbone.body.layer1.0.conv2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %backbone.body.layer1.0.conv3.weight : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %backbone.body.layer1.0.downsample.0.weight : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %backbone.body.layer1.1.conv1.weight : Float(64, 256, 1, 1, strides=[256, 1, 1, 1], requires_grad=0, device=cpu),
      %backbone.body.layer1.1.conv2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %backbone.body.layer1.1.conv3.weight : Float(256, 64, 1, 1, strides=[6

exported as onnx!


In [15]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(r'C:\Users\ureff\Documents\Spinntech\OD_TF\image1.jpg')
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

In [39]:
import onnx

# Load the ONNX model
model = onnx.load("fasterrcnn_resnet50_fpn.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %image.1[FLOAT, 3x1200x800]
) initializers (
  %backbone.body.conv1.weight[FLOAT, 64x3x7x7]
  %backbone.body.layer1.0.conv1.weight[FLOAT, 64x64x1x1]
  %backbone.body.layer1.0.conv2.weight[FLOAT, 64x64x3x3]
  %backbone.body.layer1.0.conv3.weight[FLOAT, 256x64x1x1]
  %backbone.body.layer1.0.downsample.0.weight[FLOAT, 256x64x1x1]
  %backbone.body.layer1.1.conv1.weight[FLOAT, 64x256x1x1]
  %backbone.body.layer1.1.conv2.weight[FLOAT, 64x64x3x3]
  %backbone.body.layer1.1.conv3.weight[FLOAT, 256x64x1x1]
  %backbone.body.layer1.2.conv1.weight[FLOAT, 64x256x1x1]
  %backbone.body.layer1.2.conv2.weight[FLOAT, 64x64x3x3]
  %backbone.body.layer1.2.conv3.weight[FLOAT, 256x64x1x1]
  %backbone.body.layer2.0.conv1.weight[FLOAT, 128x256x1x1]
  %backbone.body.layer2.0.conv2.weight[FLOAT, 128x128x3x3]
  %backbone.body.layer2.0.conv3.weight[FLOAT, 512x128x1x1]
  %backbone.body.layer2.0.downsample.0.weight[FLOAT, 512x256x1x1]
  %backbone.body.layer2.1.conv1.weight[FLOAT, 128x512x1

In [40]:
import numpy
import onnxruntime as rt
import cv2
import numpy as np

img = cv2.imread(r'C:\Users\ureff\Documents\Spinntech\OD_TF\image1.jpg')
img = np.dot(img[...,:3], [0.299, 0.587, 0.114])
X_test = cv2.resize(img, dsize=(1200, 800), interpolation=cv2.INTER_AREA)
X_test.resize((3, 1200, 800))

sess = rt.InferenceSession("fasterrcnn_resnet50_fpn.onnx")
input_name = sess.get_inputs()[0].name
pred_onx = sess.run(None, {input_name: X_test.astype(numpy.float32)})[0]
print(pred_onx)

[]


In [42]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [43]:
alex_model = torch.hub.load('pytorch/vision:v0.9.0', 'alexnet', pretrained=True)
alex_model

Using cache found in C:\Users\ureff/.cache\torch\hub\pytorch_vision_v0.9.0
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to C:\Users\ureff/.cache\torch\hub\checkpoints\alexnet-owt-4df8aa71.pth


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [48]:
onnx.helper.printable_graph(model.graph)

AttributeError: 'FasterRCNN' object has no attribute 'graph'